# Medical consultant case study

- Average complication rate: 10%
- Consultant complication rate: p' = 3/62 = 4.84% (is used to estimate p)

- The parameter is p: the true probability of a complication for a client of the medical consultant. 

## Setup

In [53]:
import pandas as pd
import numpy as np
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Variability of the statistic

### Sampling with replacment

By sampling with replacement from the dataset (a process called bootstrapping), the variability of the possible p' values can be approximated.



In [80]:
# number of people
n = 62 
# probability
p = 3/62

np.random.seed(0)

# generate 10000 bootstrap simulations
complications = np.random.binomial(n, p, 10000)

df = pd.DataFrame({"comp": complications})
df['comp_rate'] = df['comp'] / n

df.head()

,comp,comp_rate
0,3,0.048387
1,4,0.064516
2,3,0.048387
3,3,0.048387
4,3,0.048387


*Note: Since we only have two outcome (complication or no complication) we use the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) to generate our data*

In [81]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
comp,10000.0,2.974500,1.689359,0.0,2.000000,3.000000,4.000000,12.000000
comp_rate,10000.0,0.047976,0.027248,0.0,0.032258,0.048387,0.064516,0.193548


In [82]:
chart = alt.Chart(df, title="10,000 bootstrapped proportions").mark_bar().encode(
    alt.X('comp_rate:Q', 
           title="Bootstrapped proportion of surgical complications",
           bin=alt.BinParams(maxbins=20)),
    alt.Y('count()',
           title="Count")
)

chart

alt.Chart(...)

### Bootstrap percentile confidence interval

In [111]:
# bootstrap 2.5 percentile proportion 
q_0025 = df['comp_rate'].quantile(0.025)
print(q_0025)
# bootstrap 97.5 percentile proportion 
q_0975 = df['comp_rate'].quantile(0.975)
q_0975

0.0


0.11290322580645161

- The bootstrap 2.5 percentile proportion is 0 and the 97.5 percentile is 0.113. 

- The result is: we are confident that, in the population, the true probability of a complication is between 0% and 11.3%.

Create visualization

In [ ]:
Option 2: 

- Calculate bootstrap percentile proportions with Altair:

In [79]:
p1 = (
    alt.Chart(df)
    .transform_quantile('comp_rate', probs=[0.025], as_=['prob', 'value'])
    .mark_rule(color='orange', strokeDash=[5, 5], strokeWidth=3)
    .encode(
        x = "value:Q"
    )
)

p2 = (
    alt.Chart(df)
    .transform_quantile('comp_rate', probs=[0.975], as_=['prob', 'value'])
    .mark_rule(color='red', strokeDash=[5, 5], strokeWidth=3)
    .encode(
        x = "value:Q"
    )
)

chart + p1 + p2


alt.LayerChart(...)

## Calculate difference

In [216]:
df_male = df[(df["gender"] == "male")]
df_female = df[(df["gender"] == "female")]

male_total = len(df_male)
female_total = len(df_female)

male_promoted = df_male['decision'].value_counts().promoted
female_promoted = df_female['decision'].value_counts().promoted

male_p = round(male_promoted/male_total, 3)
female_p = round(female_promoted/female_total, 3)

p_diff_ob = round(male_p - female_p, 3)

df_p_diff_ob = pd.DataFrame({'p_diff': [p_diff_ob] })
df_p_diff_ob

,p_diff
0,0.292


## Randomization

### Crosstable

Example with only one randomization:

In [160]:
df['gender_random'] = df['gender'].sample(frac=1, random_state=123).reset_index(drop=True)

Since the randomization of files in this simulation is independent of the promotion decisions, any difference in promotion rates is due to chance.

In [161]:
pd.crosstab(df.gender_random, df.decision,  margins=True)

decision,not promoted,promoted,All
gender_random,,,
female,7,17,24
male,6,18,24
All,13,35,48


### Calculate difference

#### One randomization

In [162]:
df_male = df[(df["gender_random"] == "male")]
df_female = df[(df["gender_random"] == "female")]

male_total = len(df_male)
female_total = len(df_female)

male_promoted = df_male['decision'].value_counts().promoted
female_promoted = df_female['decision'].value_counts().promoted

male_p = round(male_promoted/male_total, 3)
female_p = round(female_promoted/female_total, 3)

p_diff = round(male_p - female_p, 3)
p_diff

0.042

#### Multiple randomizations

In [188]:
# create an empty list
random_difference = []

# make 100 randomizations and save results
for i in range(0, 100):

    df['gender_random'] = df['gender'].sample(frac=1, random_state=i).reset_index(drop=True)

    df_male = df[(df["gender_random"] == "male")]
    df_female = df[(df["gender_random"] == "female")]

    male_total = len(df_male)
    female_total = len(df_female)

    male_promoted = df_male['decision'].value_counts().promoted
    female_promoted = df_female['decision'].value_counts().promoted

    male_p = round(male_promoted/male_total, 3)
    female_p = round(female_promoted/female_total, 3)

    p_diff = round(male_p - female_p, 3)

    random_difference.append(p_diff)

In [189]:
# create pandas dataframe
df_random = pd.DataFrame({'p_diff': random_difference})


In [190]:
df_random.describe().T

,count,mean,std,min,25%,50%,75%,max
p_diff,100.0,-0.01501,0.124152,-0.292,-0.125,-0.042,0.042,0.292


In [221]:
chart1 = alt.Chart(df_random).mark_circle(size=100).transform_window(
    id='rank()',
    groupby=['p_diff']
).encode(
    alt.X('p_diff:O', title='Differences in promotion rates (male - female) across 100 shuffles'),
    alt.Y('id:O',
          axis=None,
          sort='descending')
).properties(height=300, width=400)


chart2 = alt.Chart(df_p_diff_ob).mark_circle(size=100).transform_window(
    id='rank()',
    groupby=['p_diff']
).encode(
    alt.X('p_diff:O'),
    alt.Y('id:O',
          axis=None,
          sort='descending'),
    color=alt.value('orange')
)

chart1 + chart2 


alt.LayerChart(...)

## Result

In [237]:
count_diff = df_random[df_random['p_diff'] >= p_diff_ob].count().p_diff

p_value = count_diff/len(df_random)
p_value

0.01


For our simulation, we get a 1% probability of obtaining a sample where ≥ 29.2% more male candidates than female candidates get promoted under the null hypothesis, 
We conclude The data provide strong evidence of sex discrimination against female candidates 
We reject the null hypothesis in favor of the alternative